## Great, now that we discussed a little let's continue

Given that the current approach utilized by the authors lacks reproducibility, we will explore an alternative method by leveraging nf-core pipelines for data analysis.

Please explain, how we will achieve reproducibility for the course  with this approach.


nf-core pipelines enable reproducible results by providing standardized, modular workflows that manage software dependencies and use containerization, ensuring consistent execution across different environments.

You have successfully downloaded 2 of the fastq files we will use in our study.

What is the next step if we want to first have a count table and check the quality of our fastq files? What is the pipeline called to do so?

In [4]:
!nextflow run nf-core/rnaseq --input './results/samplesheet/samplesheet.csv' --outdir './rnaseq_results'  --genome 'GRCm38' -profile docker

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Nextflow 24.04.4 is available - Please consider updating your version to it
N E X T F L O W  ~  version 23.10.1
Launching `https://github.com/nf-core/rnaseq` [astonishing_shockley] DSL2 - revision: 4053b2ec17 [master]
WARN: Access to undefined parameter `monochromeLogs` -- Initialise it to a default value eg. `params.monochromeLogs = some_value`


------------------------------------------------------
                                        ,--./,-.
        ___     __   __   __   ___     /,-._.--~'
  |\ | |__  __ /  ` /  \ |__) |__         }  {
  | \| |       \__, \__/ |  \ |___     \`-._,-`-,
                                        `._,._,'
  nf-core/rnaseq v3.15.1-g4053b2e
------------------------------------------------------
Core Nextflow options
  revision        : master
  runName         : astonishing_shockley
  containerEngine : docker
 

Analyze the 2 files using an nf-core pipeline.

What does this pipeline do?

Which are the main tools that will be used in the pipeline?

The pipeline takes a samplesheet and FASTQ files as input, performs quality control, trimming and (pseudo-)alignment, and produces a gene expression matrix and extensive QC report [[Source]](https://nf-co.re/rnaseq/3.15.1/).

The main tools that are used in the pipeline are `FastQC`, `UMI-tools`, `Trim Galore!`, `BBsplit`, `SortMeRNA`, `STAR`, `Salmon`, `HiSAT2`, `SAMtools`, `picard MarkDuplicates`, `StringTie`, `BEDTools`, `MultiQC`

As all other nf-core pipelines, the chosen pipeline takes in a samplesheet as input.

Use Python and pandas to create the samplesheet for your 2 samples. Feel free to make use of the table you created earlier today.

Choose your sample names wisely, they must be the connection of the results to the metadata. If you can't find the sample in the metadata later, the analysis was useless.

In [ ]:
!nextflow run nf-core/rnaseq --input './results/samplesheet/samplesheet.csv' --outdir './rnaseq_results'  --genome 'GRCm38' -profile docker

Explain all the parameters you set and why you set them in this way.



- `--input`: The input samplesheet which includes the sample metadata.
- `--outdir`: The output directory where the results will be saved.
- `--genome`: The name of the iGenomes reference.
- `-profile`: This instructs Nextflow to execute jobs locally, with Docker (or Singularity) to fulfill software dependencies.

## Browsing the results

How did the pipeline perform?

The process was memory and CPU-intensive, leading to a long runtime (It could not be run on my computer, we received the results from the tutors).

Explain the quality control steps. Are you happy with the quality and why. If not, why not.

Overall, the data quality appears high, with reads showing strong Phred quality scores, indicating successful sequencing. However, two samples failed the strandedness check.

Please give additional information on : 
- ribosomal rRNA: Inadequate removal of rRNA results in a lower number of useful reads (we are interested in mRNA, not rRNA, to determine gene expression levels), affecting the depth of transcript analysis. SNI_Sal_2 and SNI_Sal_4 have large percentages of rRNA (22.1% and 25.1%), which is not ideal.
- Duplication: Duplication levels can signal issues in library preparation, such as over-amplification during PCR. High duplication rates can skew the quantification of transcript abundance. The trimmed sequence duplication levels indicate that 19 out of the 32 samples failed due to high levels of sequence duplication. Sharp increases at the far right of the FastQC duplication plot may point to the enrichment of particular sequence subsets.
- GC content: Deviations from the expected normal distribution of the GC content could indicate technical biases, such as contamination or issues during sequencing. When we look at the per sequence GC content of the trimmed sequences, we see that 14 out of 32 samples failed the quality check because the GC content has a second peak at around 85% GC content.

What are the possible steps that could lead to poorer results?
- Poor rRNA removal could leave too many non-informative reads.
- High duplication rates may suggest over-amplification, reducing data complexity.
- Skewed GC content distributions might point to contamination or biases in the sequencing process.

Would you exclude any samples? If yes, which and why?

I would think of excluding SNI_Sal_2 and SNI_Sal_4 due to their rRNA contamination and high levels of duplication. They have very high distance to all other samples indicating that there may be something wrong with them (STAR_SALMON DESeq2 sample similarity). These two samples are also very far away from all other points in the PCA plot (STAR_SALMON DESeq2 PCA plot).

We don't exclude these samples now, we proceed with the analysis, but we keep in mind that these samples might be corrupted.

What would you now do to continue the experiment? What are the scientists trying to figure out? Which packages on R or python would you use?

The next step would be Differential Expression Analysis. I would use DESeq2 through the rpy2 package.